# Manual of root_segmentor_VIB

### Sep 0: imports

Import the required modules

In [1]:
import os
import sys
sys.path.append("..")
import root_segmentor_VIB as rs
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib tk


### Step 1: look at some sample data

The method `load_training_image` allows to read an image and the tracings for training as well. The `auto_transform` option allows to automatically transform the tracings coordinates into the coordinate system of the image.

In [2]:
im, names, vertices_s, vertices_e = rs.load_training_image(img_file = "../sample_uniform_1/EOS 550D_042.JPG",
                                                        root_traces_file = "../sample_uniform_1/EOS 550D_042 vertices.csv",
                                                        auto_transform=True)

#transform into row-column coordinates
vertices_s_RC = rs.flip_XY_RC(vertices_s)
vertices_e_RC = rs.flip_XY_RC(vertices_e)

To create training data from the loaded images, the tracings are first transformed into a root-mask with `create_root_mask` after which the function `create_root_buffer_background_image` transforms this into an image containing root-pixels (1), relevant background pixels for making a classification (2) and unclassified pixels (3). To do that, buffer zones are used around the images.

Lastly, the function `create_training_data` finally selects the left half of the image for training purposes. Can be skipped if entire image should be used for training.

In [3]:
# create root mask
root_mask = rs.create_root_mask(im, vertices_s_RC, vertices_e_RC)
root_buffer_background = rs.create_root_buffer_background_image(root_mask)

# create training data labels
training_labels = rs.create_training_data(root_buffer_background)

The function `show_traces` allows to plot an image with the traincing on top (similar to imshow). Use `%matplotlib tk` for pop-up viewer

In [4]:
plt.subplot(1, 3, 1)
rs.show_traces(vertices_s, vertices_e, im)
plt.subplot(1, 3, 2)
rs.show_traces(vertices_s, vertices_e, root_buffer_background)
plt.subplot(1, 3, 3)
rs.show_traces(vertices_s, vertices_e, training_labels)

### Step 2: Compile a dataset for training

The tracings of multiple images are combined to learn a pixel-classifier. To achieve this goal, the following steps are taken:
* All images and tracings in `some_folder` are listed
* Per image, pixel-level features are computed (texture, gradient image etc.)
* The per image, the function `create_root_buffer_background_image` is used to comput the label of every pixel
* A fraction is points is sampled (reducing training dataset size and rebalancing it somewhat)
* The preveous steps are applied to all images in `some_folder` and combined in a features dataset `X` and labels dataset `Y`

The first step only computes labels and features per image and stores them as `npy` files.

In [ ]:

# compute FEATURES and LABELS for each image in a given folder
files_list = ['../sample_uniform_1/'+f for f in os.listdir('../sample_uniform_1') if f[-3:] == "JPG"]
rs. imgs_to_XY_data(files_list)

The second step combines the generated files to create `X` and `Y`

In [6]:
# create training datasets
features_file_list = ['../sample_uniform_1/'+f for f in os.listdir('../sample_uniform_1') if f[-3:] == "npy" and "FEATURES" in f]
X, Y = rs.compile_training_dataset_from_precomputed_features(features_file_list)

### Step 3: Train a model and save it

The compiled dataset is used to train a random forest classifier

In [17]:
# fit random forest classifier
clf = rs.RandomForestClassifier(n_estimators=100, n_jobs=-1,
                            max_depth=10, max_samples=0.05)
clf.fit(X, Y)
# dump the model to a file
# dump(clf, '../models/RF_550D_042_078_NTrees-100.joblib')

RandomForestClassifier(max_depth=10, max_samples=0.05, n_jobs=-1)

### Step 4: Load a saved model

Select a saved model and load it

In [2]:
clf = rs.load('../models/RF_550D_042_078_NTrees-100.joblib')

### Step 5: Load new image to make predictions (and compare with tracings)

In [8]:
im = rs.io.imread("../sample_uniform_1/test_cases/EOS 550D_082.JPG")
# compute features
features = rs.im2features(im)
# predict
predicted_segmentation = rs.predict_segmentor(clf, features)
# clean detected roots
roots = rs.clean_predicted_roots(predicted_segmentation)

Visualize the results

In [15]:
# draw detected roots
rs.draw_detected_roots(roots, im)

### Step 6: Export the results to a file

The lenths, orientation, position etc. of the roots can be exported to a file

In [10]:
results_df = rs.measure_roots(roots)
results_df.to_excel("../sample_results/measurements.xlsx")

### Step 7: Automate classification per folder

Lists all files in `some_dir`, detects roots and saves the results in a xlsx file. All detected roots are saved for quality checking.

In [ ]:
# list all test images in folder
features_file_list = ['../sample_uniform_1/test_cases2/'+f for f in os.listdir('../sample_uniform_1/test_cases2') if f[-3:] == "JPG"]

all_results = []

for fname in features_file_list:
    # read image
    im = rs.io.imread(fname)
    # compute features
    features = rs.im2features(im)
    # predict
    predicted_segmentation = rs.predict_segmentor(clf, features)
    # clean detected roots
    roots = rs.clean_predicted_roots(predicted_segmentation)
    # compute root properties
    results_df = rs.measure_roots(roots)
    results_df["fname"] = fname
    # append to results list
    all_results.append(results_df)
    # save image for quality check
    rs.save_detected_roots_im(roots, im, fname[:-4] + "result.png")


pd.concat(all_results).to_excel("../sample_results/measurements.xlsx")
